In [ ]:
!nvidia-smi

# 匯入函式庫

In [ ]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm


# 匯入資料與前處理

In [ ]:
diff_log_data = pd.read_csv('./recursive_linear_residue.csv', index_col=0)
diff_log_data = diff_log_data.astype('float64')
diff_log_data.index = pd.to_datetime(diff_log_data.index)

diff_log_data

# CPU/GPU、自定義資料集、模型、訓練函數

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def SetSeed(myseed):
    # Python random module
    random.seed(myseed)
    # Numpy
    np.random.seed(myseed)
    # Torch
    torch.manual_seed(myseed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(myseed)
        torch.cuda.manual_seed_all(myseed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X, WindowSize):
        X = np.expand_dims(X, 1)
        self.X = X.astype(np.float64)
        self.WindowSize = WindowSize
        
    def __len__(self):
        return len(self.X) - self.WindowSize

    def __getitem__(self, idx):
        return (self.X[idx:idx+self.WindowSize], self.X[idx+self.WindowSize])
        # return (X = [seqs, features], y)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, num_layers, hidden_size):
        super().__init__()
        
        self.Input_HiddenLayer = nn.LSTM(input_size=1, hidden_size=hidden_size, num_layers=num_layers)

        self.OutputLayer = nn.Linear(hidden_size, 1)

    def forward(self, input):
        # input.shape = [BatchSize, WindowSize, 1]
        input = input.permute(1, 0, 2)
        # input.shape = [WindowSize, BatchSize, 1]
        hidden, _ = self.Input_HiddenLayer(input)
        # hidden.shape = [WindowSize, BatchSize, HiddenSize]
        hidden = hidden[-1]
        # hidden.shape = [BatchSize, HiddenSize]
        output = self.OutputLayer(hidden)
        
        return output

In [ ]:
def train_under_config_and_evaluating_at_num_epochs_list(forex_data,
                                                         length_input_sequence,
                                                         num_epochs_list,
                                                         num_hidden_layers,
                                                         num_hidden_sizes,
                                                         batch_sizes,
                                                         device):
    '''
    forex_data,
    length_input_sequence,
    num_epochs_list,
    learning_rate,
    num_hidden_layers,
    num_hidden_sizes,
    batch_sizes,
    device
    '''
    # setseed
    SetSeed(9527)
    
    # dataset_train
    training_data = forex_data.loc['1981-01-01':'2008-12-31']
    training_dataset = TimeSeriesDataset(training_data, length_input_sequence)
    # dataset_valid
    validation_start_index = len(forex_data.loc['1981-01-01':'2008-12-31']) - length_input_sequence
    validation_end_index = len(forex_data.loc['1981-01-01':'2016-12-31'])
    validation_data = forex_data[validation_start_index:validation_end_index]
    validation_dataset = TimeSeriesDataset(validation_data, length_input_sequence)
    
    # dataloader_train
    training_dataloader = DataLoader(training_dataset, batch_size=batch_sizes, shuffle=True)
    # dataloader_valid
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_sizes, shuffle=False)
    
    # model
    model = LSTM(num_hidden_layers, num_hidden_sizes).double()
    # criterion & optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters())
    
    # training & evaluating
    min_valid_loss_at_best_epoch = 100000
    min_valid_loss_epoch = 0
    for epoch in tqdm(range(num_epochs_list[-1])):
        # training
        model.to(device)
        model.train()
        for X, y in training_dataloader:
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            ypred = model(X)
            loss = criterion(ypred, y)
            loss.backward()
            optimizer.step()
            
        # evaluating
        if (epoch + 1) in num_epochs_list:
            model.eval()
            valid_loss = 0
            len_valid = 0
            for X, y in  validation_dataloader:
                len_valid += len(X)
                X, y = X.to(device), y.to(device)
                with torch.no_grad():
                    ypred = model(X)
                    loss = criterion(ypred, y)
                valid_loss += loss.detach().cpu().item() * len(X)
            valid_loss = valid_loss / len_valid
            if valid_loss < min_valid_loss_at_best_epoch:
                min_valid_loss_at_best_epoch = valid_loss
                min_valid_loss_epoch = epoch + 1
    
    return min_valid_loss_epoch, min_valid_loss_at_best_epoch

In [ ]:
def find_optimum_config_under_specific_input_length(forex_data,
                                                    length_input_sequence,
                                                    num_epochs_list,
                                                    num_hidden_layers_list,
                                                    num_hidden_sizes_list,
                                                    batch_sizes_list,
                                                    device=device):
    min_valid_loss = 100000
    min_valid_config_under_specific_input_length = ()
    for num_hidden_layers in num_hidden_layers_list:
        for num_hidden_sizes in num_hidden_sizes_list:
            for batch_sizes in batch_sizes_list:
                print('\nTraining under config:', (num_hidden_layers, num_hidden_sizes, batch_sizes))
                min_valid_loss_epoch, min_valid_loss_at_best_epoch = train_under_config_and_evaluating_at_num_epochs_list(forex_data,
                                                                                                                          length_input_sequence,
                                                                                                                          num_epochs_list,
                                                                                                                          num_hidden_layers,
                                                                                                                          num_hidden_sizes,
                                                                                                                          batch_sizes,
                                                                                                                          device)
                if min_valid_loss_at_best_epoch < min_valid_loss:
                    min_valid_loss = min_valid_loss_at_best_epoch
                    min_valid_config_under_specific_input_length = (num_hidden_layers, num_hidden_sizes, batch_sizes, min_valid_loss_epoch)

                    print('\nvalid_loss improve to',
                            min_valid_loss,
                            'under config:',
                            min_valid_config_under_specific_input_length,
                            '(num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)')

    print('\nmin valid loss config under specific input length',
          length_input_sequence,
          'is:',
          min_valid_config_under_specific_input_length,
          '(num_hidden_layers, num_hidden_sizes, batch_sizes, num_epochs)',
          'and the valid loss is:',
          min_valid_loss)
    return min_valid_config_under_specific_input_length, min_valid_loss

# cad

In [ ]:
cad_data = diff_log_data.iloc[:,0]
print(cad_data.loc['1981-01-01':'2008-12-31'])
print(cad_data.loc['2009-01-01':'2016-12-31'])
print(cad_data.loc['2017-01-01':'2020-12-31'])

In [ ]:
# random walk mse(validation)
se = cad_data.loc['2009-01-01':'2016-12-31'] ** 2
mse = sum(se) / len(se)
mse

In [ ]:
# configuration on length_input_sequence = 5
length_input_sequence = 5
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(cad_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )

In [ ]:
# configuration on length_input_sequence = 10
length_input_sequence = 10
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(cad_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )

In [ ]:
# configuration on length_input_sequence = 20
length_input_sequence = 20
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(cad_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )

In [ ]:
# configuration on length_input_sequence = 60
length_input_sequence = 60
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(cad_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )

# aud

In [ ]:
aud_data = diff_log_data.iloc[:,1]
print(aud_data.loc['1981-01-01':'2008-12-31'])
print(aud_data.loc['2009-01-01':'2016-12-31'])
print(aud_data.loc['2017-01-01':'2020-12-31'])

In [ ]:
# random walk mse(validation)
se = aud_data.loc['2009-01-01':'2016-12-31'] ** 2
mse = sum(se) / len(se)
mse

In [ ]:
# configuration on length_input_sequence = 5
length_input_sequence = 5
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(aud_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )

In [ ]:
# configuration on length_input_sequence = 10
length_input_sequence = 10
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(aud_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )

In [ ]:
# configuration on length_input_sequence = 20
length_input_sequence = 20
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(aud_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )

In [ ]:
# configuration on length_input_sequence = 60
length_input_sequence = 60
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(aud_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )

# gbp

In [ ]:
gbp_data = diff_log_data.iloc[:,2]
print(gbp_data.loc['1981-01-01':'2008-12-31'])
print(gbp_data.loc['2009-01-01':'2016-12-31'])
print(gbp_data.loc['2017-01-01':'2020-12-31'])

In [ ]:
# random walk mse(validation)
se = gbp_data.loc['2009-01-01':'2016-12-31'] ** 2
mse = sum(se) / len(se)
mse

In [ ]:
# configuration on length_input_sequence = 5
length_input_sequence = 5
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(gbp_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )

In [ ]:
# configuration on length_input_sequence = 10
length_input_sequence = 10
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(gbp_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )

In [ ]:
# configuration on length_input_sequence = 20
length_input_sequence = 20
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(gbp_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )

In [ ]:
# configuration on length_input_sequence = 60
length_input_sequence = 60
num_epochs_list = [5, 10, 15, 20, 25, 30]
num_hidden_layers_list = [1, 2, 3, 4]
num_hidden_sizes_list = [25, 50, 100, 200, 400, 800, 1600]
batch_sizes_list = [16, 32, 64, 128, 256]

In [ ]:
find_optimum_config_under_specific_input_length(gbp_data,
                                                length_input_sequence,
                                                num_epochs_list,
                                                num_hidden_layers_list,
                                                num_hidden_sizes_list,
                                                batch_sizes_list
                                                )